<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nni

In [ ]:
import nni
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_filters=32, kernel_size=3):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, num_filters, kernel_size)
        self.conv2 = nn.Conv2d(num_filters, num_filters, kernel_size)
        self.fc1 = nn.Linear(num_filters * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training loop
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item()}')

# Hyperparameter tuning
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor()),
        batch_size=64, shuffle=True)

    params = nni.get_next_parameter()
    model = SimpleCNN(num_filters=params['num_filters'], kernel_size=params['kernel_size']).to(device)
    optimizer = optim.SGD(model.parameters(), lr=params['lr'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, 10):
        train(model, device, train_loader, optimizer, criterion, epoch)

    acc = test(model, device, test_loader)
    nni.report_final_result(acc)

if __name__ == '__main__':
    main()

In [ ]:
experimentName: SimpleCNN
trialCommand: python3 your_script.py
trialCodeDirectory: .
trialConcurrency: 1
maxExecDuration: 1h
maxTrialNum: 10
trainingService:
  platform: local
  useActiveGpu: false

searchSpace:
  num_filters:
    _type: choice
    _value: [16, 32, 64]
  kernel_size:
    _type: choice
    _value: [3, 5]
  lr:
    _type: choice
    _value: [0.01, 0.001, 0.0001]

tuner:
  name: TPE
  classArgs:
    optimize_mode: minimize